# Traffic Application Testing Code

This notebook is designed to communicate with the Traffic Application server and perform a series of tests. It is necessary to restart the server for each test to ensure that the test did not fail because of the state the server was in due to the previous tests. Each time you restart the server ensure that it begins with a database in a known state.

In [1]:
# The modules needed
import random
import requests
import shutil
import json
import subprocess

In [2]:
# The port the server should run on

server_port = '8081'

In [3]:
# The core code

def request2server(url,cookies):
    ucookie = cookies[0]
    mcookie = cookies[1]
    try:
        r = requests.get(url,cookies=dict(u_cookie=cookies[0],m_cookie=cookies[1]),timeout=15)
        for c in r.cookies:
            if (c.name == 'u_cookie'):
                ucookie = c.value
            if (c.name == 'm_cookie'):
                mcookie = c.value
        act=json.loads(r.text)
        return [[ucookie,mcookie],act]
    except :
        print("Invalid login")
        return [[ucookie,mcookie],[]]
    
def requestfilefromserver(url,cookies):
    ucookie = cookies[0]
    mcookie = cookies[1]
    try:
        r = requests.get(url,cookies=dict(u_cookie=cookies[0],m_cookie=cookies[1]),timeout=15)
        for c in r.cookies:
            if (c.name == 'u_cookie'):
                ucookie = c.value
            if (c.name == 'm_cookie'):
                mcookie = c.value
        return [[ucookie,mcookie],r.text]
    except :
        print("Invalid login")
        return [[ucookie,mcookie],[]]
    
def find_redirect(act):
    for a in act:
        if 'type' in a:
            if a['type'] == 'redirect':
                if 'where' in a:
                    return a['where']
    return None

def find_refill(act,where):
    for a in act:
        if 'type' in a:
            if a['type'] == 'refill':
                if 'where' in a:
                    if a['where'] == where:
                        return a['what']
    return None

def randparam():
    return "&randn="+str(random.randrange(10000,99999))

def do_login(cookies,user,pasw):
    return request2server("http://localhost:"+server_port+"/action?command=login&usernameinput="+user+"&passwordinput="+pasw+randparam(),cookies)

def do_add(cookies,loc,occupancy,vtype):
    return request2server("http://localhost:"+server_port+"/action?command=add&locationinput="+loc+"&occupancyinput="+str(occupancy)+"&typeinput="+vtype+randparam(),cookies)

def do_undo(cookies,loc,occupancy,vtype):
    return request2server("http://localhost:"+server_port+"/action?command=undo&locationinput="+loc+"&occupancyinput="+str(occupancy)+"&typeinput="+vtype+randparam(),cookies)

def do_summary(cookies):
    return request2server("http://localhost:"+server_port+"/action?command=summary"+randparam(),cookies)

def do_logout(cookies):
    return request2server("http://localhost:"+server_port+"/action?command=logout"+randparam(),cookies)

def do_file_fetch(cookies,file):
    return requestfilefromserver("http://localhost:"+server_port+"/"+file,cookies)

In [4]:
def checked_login(test,cookies,user,pasw):
    [cookies,act] = do_login(cookies, user,pasw)
    where = find_redirect(act)
    if(where == None):
        print("Test "+str(test)+" Failed - Expected redirect during login.")
        return ['',act,False]
    if (where != '/page.html'):
        print("Test "+str(test)+" Failed - Expected /page.html got {"+where+"}")
        return [cookies,act,False]
    return [cookies,act,True]

def checked_logout(test,cookies):
    [cookies,act] = do_logout(cookies)
    where = find_redirect(act)
    if (where != '/index.html'):
        print("Test "+str(test)+" Failed - Expected /index.html got {"+where+"}")
        return [cookies,act,False] 
    
    return [cookies,act,True]

In [5]:
# Test 1 - Simple login
def test1():
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(1,cookies,"test1","password1")
        where = find_redirect(act)
        if flag!=True:
            return 0

        [cookies,act,flag] = checked_logout(1,cookies)        
        where = find_redirect(act) 
        if flag!=True:
            return 0
        
        print("Test 1 Passed")
        return 1
    except:
        print("Test 1 Failed - Exception Caused.")
        return 0


In [6]:
# Test 2 - Bad login
def test2():
    try:
        cookies = ['','']
        [cookies,act2] = do_login(cookies,"test1","passwor1")
        where = find_redirect(act2)
        if (where != None):
            print("Test 2 Failed - Unexpected redirect.")
            return 0
        
        message = find_refill(act2,'message')
    
        if (message == None):
            print("Test 2 Failed - No failure message present")
            return 0

        [cookies,act,flag] = checked_logout(2,cookies)
        
        where = find_redirect(act) 
        if flag!=True:
            return 0
        
        print("Test 2 Passed")
        return 1
    except:
        print("Test 2 Failed - Exception Caused.")
        return 0        

In [7]:
# Test 4 - Simple add
def test4():
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(4,cookies,"test1","password1")
        where = find_redirect(act)
        if flag!=True:
            return 0
        
        [cookies,act] = do_add(cookies,"road",1,"car")
      
        where = find_redirect(act)
    
        if (where != None):
            print("Test 4 Failed - Unexpected redirect.(2)")
            return 0
        
        message = find_refill(act,'message')
    
        if (message == None):
            print("Test 4 Failed - No acknowledge message present")
            return 0
        
        total = find_refill(act,'total')
    
        if (total == None):
            print("Test 4 Failed - No total message present")
            return 0
        
        if (total != "1"):
            print("Test 4 Failed - Total not expected value")
            return 0
         
        [cookies,act,flag] = checked_logout(4,cookies)        
        where = find_redirect(act) 
        if flag!=True:
            return 0
        
        print("Test 4 Passed")
        return 1
    except:
        print("Test 4 Failed - Exception Caused.")
        return 0

In [8]:
# Test 8 - Unknown vehicle type, step 1
def test8():
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(8,cookies,"test1","password1")
        where = find_redirect(act)
        if flag!=True:
            return 0
    
        [cookies_x,act] = do_add(cookies,"road",1,"caravan")
        where = find_redirect(act)

        if (where != None):
            print("Test 8 Failed - Unexpected redirect.(2)")
            return 0
        
        message = find_refill(act,'message')
        
        if (message == None):
            print("Test 8 Failed - No acknowledge message present(3)")
            return 0
        
        total = find_refill(act,'total')
    
        if (total != None):
            if (total != "0"):
                print("Test 8 Failed - Total not expected value, should be zero.")
                return 0
        
        [cookies,act,flag] = checked_logout(8,cookies)        
        where = find_redirect(act) 
        if flag!=True:
            return 0

        print("Test 8 Passed")
        return 1
    except:
        print("Test 8 Failed - Exception Caused.")
        return 0

In [9]:
# Test 10 - Simple undo
def test10():
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(10,cookies,"test1","password1")
        where = find_redirect(act)
        if flag!=True:
            return 0
        
            
        [cookies_x,act] = do_add(cookies,"road",1,"car")
        
        where = find_redirect(act)

        if (where != None):
            print("Test 10 Failed - Unexpected redirect.(2)")
            return 0
        
        total = find_refill(act,'total')
        
        if (total != None):
            if (total != "1"):
                print("Test 10 Failed - Total not expected value, should be 1. Was {"+total+"}")
                return 0
        
        [cookies_x,act] = do_undo(cookies,"road",1,"car")
      
        where = find_redirect(act)
    
        if (where != None):
            print("Test 10 Failed - Unexpected redirect.(2)")
            return 0
        total = find_refill(act,'total')
        
        if (total == None):
            print("Test 10 Failed - Total not found.")
            return 0
    
        if (total != "0"):
            print("Test 10 Failed - Total not expected value, should be 0. Was {"+total+"}")
            return 0
        
        [cookies,act,flag] = checked_logout(10,cookies)        
        where = find_redirect(act) 
        if flag!=True:
            return 0
        
        print("Test 10 Passed")
        return 1
    except:
        print("Test 10 Failed - Exception Caused.")
        return 0

In [10]:
# Test 23 - two users sessions should not overlap totals
def test23():
    try:
        cookies1 = ['','']
        [cookies1,act1,flag1] = checked_login(23,cookies1,"test1","password1")
        if flag1!=True:
            return 0
        
        cookies2 = ['','']
        [cookies2,act2,flag2] = checked_login(23,cookies2,"test2","password2")
        if flag2!=True:
            return 0
        
        [cookies1,act] = do_add(cookies1,"road",1,"car")
        
        where = find_redirect(act)
    
        if (where != None):
            print("Test 23 Failed - Unexpected redirect.")
            return 0
        
        message = find_refill(act,'message')
    
        if (message == None):
            print("Test 23 Failed - No acknowledge message present")
            return 0
        
        total = find_refill(act,'total')
    
        if (total == None):
            print("Test 23 Failed - No total message present")
            return 0
        
        if (total != "1"):
            print("Test 23 Failed - Total not expected value 1 was "+total)
            return 0

        [cookies2,act] = do_add(cookies2,"road",1,"bus")
            
        where = find_redirect(act)
    
        if (where != None):
            print("Test 23 Failed - Unexpected redirect.")
            return 0
        
        message = find_refill(act,'message')
    
        if (message == None):
            print("Test 23 Failed - No acknowledge message present")
            return 0
        
        total = find_refill(act,'total')
    
        if (total == None):
            print("Test 23 Failed - No total message present")
            return 0
        
        if (total != "1"):
            print("Test 23 Failed - Total not expected value 1 was "+total)
            return 0
        
        [cookies2,act] = do_add(cookies2,"road",1,"bicycle")
            
        where = find_redirect(act)
    
        if (where != None):
            print("Test 23 Failed - Unexpected redirect.")
            return 0
        
        message = find_refill(act,'message')
    
        if (message == None):
            print("Test 23 Failed - No acknowledge message present")
            return 0
        
        total = find_refill(act,'total')
    
        if (total == None):
            print("Test 23 Failed - No total message present")
            return 0
        
        if (total != "2"):
            print("Test 23 Failed - Total not expected value 2 was "+total)
            return 0

        [cookies1,act1,flag1] = checked_logout(23,cookies1)        
        where1 = find_redirect(act1) 
        if flag1!=True:
            return 0
        
        [cookies2,act2,flag2] = checked_logout(23,cookies2)        
        where1 = find_redirect(act1) 
        if flag1!=True:
            return 0

        print("Test 23 Passed")
        return 1
    except:
        print("Test 23 Failed - Exception Caused.")
        return 0

In [11]:
# Test 25 - Fetch the hours summary - not a real test, just shows the process
def test25():
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(25,cookies,"test1","password1")
        where = find_redirect(act)
        if flag!=True:
            return 0
        
        [cookies,content] = do_file_fetch(cookies,"/statistics/hours.csv")
        
        print(content)
        
        [cookies,act,flag] = checked_logout(25,cookies)        
        where = find_redirect(act) 
        if flag!=True:
            return 0

        print("Test 25 Passed")
        return 1
    except:
        print("Test 25 Failed - Exception Caused.")
        return 0

In [12]:
tests = [('Test 1', test1, 'db/clean.db'),('Test 2', test2, 'db/clean.db'),
         ('Test 4', test4, 'db/clean.db'),('Test 8', test8, 'db/clean.db'),
         ('Test 10', test10, 'db/clean.db'),('Test 23', test23, 'db/clean.db'),
         ('Test 25', test25, 'db/clean.db')]

mark_total = 0

for test in tests:
    print("Running"+test[0])
    try:
        shutil.copy(test[2],'traffic.db')
        sp = subprocess.Popen('python.exe server.py '+server_port,stdout=subprocess.DEVNULL)
        mark_total += test[1]()
        print("Test Finished")
        sp.terminate()
    except:
        try:
            sp.terminate()
        except:
            pass
        print("Test Process Generated Exception")
        
print("Marks = {:d}/7".format(mark_total))


RunningTest 1
Test 1 Passed
Test Finished
RunningTest 2
Test 2 Passed
Test Finished
RunningTest 4
Test 4 Passed
Test Finished
RunningTest 8
Test 8 Passed
Test Finished
RunningTest 10
Test 10 Passed
Test Finished
RunningTest 23
Test 23 Passed
Test Finished
RunningTest 25
Username,Day,Week,Month
test1,0.0,0.0,0.0

Test 25 Passed
Test Finished
Marks = 7/7
